# Prepare input Data 
## split original ECG signal into frames

In [1]:
val WINDOW = 40   // samples per window
val STEP = 2
val SAMPLES = 200000
val samplesPerSecond = 100

val scale = 1/200.0
val inputDataFile = "a02.dat"
val WindowsPerFrame = 150

val samplesPerFrame = WindowsPerFrame * WINDOW


In [2]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

In [70]:
// Databricks

//dbutils.fs.mkdirs("/FileStore/fxo/ted")
//dbutils.fs.ls("FileStore")

//val rootDir = "FileStore/fxo/ted/"
//val fileName = "/dbfs/" + rootDir +"a02.dat"



Name: Syntax Error.
Message: 
StackTrace: 

In [3]:
// GCP

val rootDir = "/home/fsainz/src/data/"
val fileName = rootDir + inputDataFile 
val secondPerFrame = samplesPerFrame / samplesPerSecond
val framesFileName = "frames-"+ inputDataFile.split('.')(0)+"_" + secondPerFrame +"s.csv"


In [4]:
println("Generating frames of " + samplesPerFrame + " samples (" + 
        + secondPerFrame + "s.) long. ")
        println("OUTPUTFILE =" + framesFileName)

Generating frames of 6000 samples (60s.) long. 
OUTPUTFILE =frames-a02_60s.csv


In [5]:


//val rawData = sc.textFile("/FileStore/tables/gqh5mjl51507524821196/a02.dat", numPartitions)
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

var frames = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]

var pos = 0
var available = 0


val frameBytes =  samplesPerFrame * 2 // read shorts 2 bytes.
val nFrames = inFile.length / frameBytes
println("Input file length =" + inFile.length  )
println("nFrames = " + nFrames)

var dataFrame = Array[Double]()
for (  w  <- 1 to nFrames.toInt ) {
   dataFrame = new Array[Double](samplesPerFrame)
    // new object is need to append at the end!!
   for ( i <- 0 until samplesPerFrame) {
 
    val value = input.readShort()   // two bytes
    dataFrame( i ) = (value.toDouble * scale) 
    
  }
  frames.append(dataFrame)
   
}
input.close()

Input file length =6364000
nFrames = 530


In [6]:
val csvFilePath = rootDir + framesFileName
// FileWriter

val file = new File(csvFilePath)
val bw = new BufferedWriter(new FileWriter(file))
var line = ""
for ( f <- frames) {
  f.foreach( x => line +=  x.toString + "," )
  bw.write(line.substring(0,line.length-1) + "\n")
  line = ""
 
}
bw.close()
println(csvFilePath + "Written")